# 🎨 SD v1.5 Complete Training on Google Colab

完整的 Stable Diffusion v1.5 微調訓練流程，專為 Google Colab 優化。

## 📋 功能特色
- 🔧 自動環境設置與依賴修復
- 🎯 支援 LoRA 高效微調
- 📊 FashionCLIP 特徵提取
- 💾 自動保存與備份
- 📈 實時訓練監控
- 🚀 多 GPU 配置優化

## ⚠️ 使用前須知
1. 確保已啟用 GPU 運行時 (Runtime > Change runtime type > GPU)
2. 建議使用 Colab Pro 以獲得更好的 GPU 和更長的運行時間
3. 準備好訓練圖片（建議 10-50 張高品質圖片）

## 🔄 執行順序
請按順序執行以下所有單元格，不要跳過任何步驟。

## 🔧 步驟 1: 環境檢查與初始化

In [ ]:
# 檢查運行環境
import sys
import os
import subprocess

print("🔍 環境檢查中...")
print(f"Python 版本: {sys.version}")
print(f"當前目錄: {os.getcwd()}")

# 檢查 GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ GPU 已啟用")
        print(result.stdout.split('\n')[8:10])  # 顯示 GPU 信息
    else:
        print("❌ GPU 未啟用，請啟用 GPU 運行時")
        print("操作: Runtime > Change runtime type > Hardware accelerator > GPU")
except:
    print("⚠️ 無法檢查 GPU 狀態")

# 檢查是否在 Colab 中
IN_COLAB = 'google.colab' in sys.modules
print(f"運行環境: {'🌐 Google Colab' if IN_COLAB else '💻 Local Environment'}")

if not IN_COLAB:
    print("⚠️ 此 Notebook 專為 Google Colab 設計")

## 📦 步驟 2: 依賴安裝與修復

In [ ]:
# 自動依賴修復腳本
print("🔧 開始依賴安裝與修復...")
print("這可能需要 5-10 分鐘，請耐心等待...")

# 卸載可能衝突的套件
conflicting_packages = [
    "sentence-transformers", 
    "transformers", 
    "torch", 
    "torchvision", 
    "torchaudio",
    "fastai"
]

print("🗑️ 清理舊版本套件...")
for package in conflicting_packages:
    try:
        !pip uninstall -y {package} -q
        print(f"   🗑️ {package}")
    except:
        pass

print("📦 安裝 PyTorch 生態系統...")
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 \
    --index-url https://download.pytorch.org/whl/cu118 --force-reinstall -q

print("📦 安裝 Transformers...")
!pip install "transformers>=4.41.0,<5.0.0" --force-reinstall -q

print("📦 安裝其他依賴...")
!pip install diffusers[torch] accelerate "peft>=0.4.0" packaging \
    matplotlib seaborn numpy pillow scikit-learn -q

print("📦 安裝 sentence-transformers...")
!pip install sentence-transformers -q

print("📦 嘗試安裝 xformers（可選）...")
try:
    !pip install xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu118 -q
    print("✅ xformers 安裝成功")
except:
    print("⚠️ xformers 安裝失敗（將使用標準注意力機制）")

print("✅ 依賴安裝完成！")

## 🔍 步驟 3: 驗證安裝

In [ ]:
# 驗證關鍵套件安裝
print("🔍 驗證安裝狀態...")

try:
    import torch
    import transformers
    import diffusers
    
    print(f"✅ torch: {torch.__version__}")
    print(f"✅ transformers: {transformers.__version__}")
    print(f"✅ diffusers: {diffusers.__version__}")
    
    # 檢查 CUDA
    if torch.cuda.is_available():
        print(f"✅ CUDA 可用: {torch.cuda.get_device_name(0)}")
        print(f"✅ CUDA 版本: {torch.version.cuda}")
        print(f"✅ GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("❌ CUDA 不可用")
    
    # 測試關鍵導入
    from diffusers import StableDiffusionPipeline
    from peft import LoraConfig
    print("✅ 關鍵套件導入測試通過")
    
    print("\n🎉 所有檢查通過，可以開始訓練！")
    
except ImportError as e:
    print(f"❌ 導入錯誤: {e}")
    print("請重新啟動運行時並重新執行前面的步驟")
    
except Exception as e:
    print(f"❌ 其他錯誤: {e}")

## 💾 步驟 4: Google Drive 設置

In [ ]:
# 掛載 Google Drive
from google.colab import drive
import os

print("📁 掛載 Google Drive...")
drive.mount('/content/drive')

# 創建工作目錄
work_dir = "/content/drive/MyDrive/SD_v1.5_Training"
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"📂 工作目錄: {work_dir}")
print(f"📍 當前目錄: {os.getcwd()}")

# 創建子目錄
subdirs = ['models', 'checkpoints', 'validation', 'logs', 'uploaded_images']
for subdir in subdirs:
    os.makedirs(subdir, exist_ok=True)
    print(f"📁 創建目錄: {subdir}")

print("✅ Google Drive 設置完成")

## 🎯 步驟 5: 載入主要訓練腳本

In [ ]:
# 如果需要，上傳主要訓練腳本
# 這裡我們直接定義核心類別和函數

print("📥 載入訓練腳本...")

# 這裡可以選擇:
# 1. 上傳 main_training_script.py 文件
# 2. 或者直接在下面的單元格中定義所需的類別

# 方法 1: 上傳文件
from google.colab import files
print("請上傳 main_training_script.py 文件:")
uploaded = files.upload()

if 'main_training_script.py' in uploaded:
    print("✅ 訓練腳本上傳成功")
    # 導入主要類別
    exec(open('main_training_script.py').read())
    print("✅ 訓練腳本載入完成")
else:
    print("❌ 未找到 main_training_script.py")
    print("請確保上傳了正確的文件")

## 📤 步驟 6: 上傳訓練圖片

In [ ]:
# 上傳訓練圖片
from google.colab import files
import os
from PIL import Image

print("📤 上傳訓練圖片...")
print("建議上傳 10-50 張高品質圖片（JPG/PNG 格式）")

uploaded = files.upload()

# 處理上傳的圖片
image_paths = []
upload_dir = "uploaded_images"

for filename, content in uploaded.items():
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        file_path = os.path.join(upload_dir, filename)
        with open(file_path, 'wb') as f:
            f.write(content)
        
        # 驗證圖片
        try:
            img = Image.open(file_path)
            img.verify()
            image_paths.append(file_path)
            print(f"✅ {filename} ({img.size})")
        except Exception as e:
            print(f"❌ {filename} 無效: {e}")
            os.remove(file_path)
    else:
        print(f"⚠️ 跳過非圖片文件: {filename}")

print(f"\n📊 成功上傳 {len(image_paths)} 張圖片")

if len(image_paths) == 0:
    print("❌ 沒有有效的圖片，請重新上傳")
else:
    print("✅ 圖片上傳完成，可以開始訓練")

## 🔧 步驟 7: 訓練配置

In [ ]:
# 訓練配置
import torch

print("🔧 配置訓練參數...")

# 檢查 GPU 類型並自動配置
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0

print(f"🎯 檢測到 GPU: {gpu_name}")
print(f"💾 GPU 記憶體: {gpu_memory:.1f} GB")

# 根據 GPU 自動配置
if "T4" in gpu_name:
    config = {
        "train_batch_size": 1,
        "gradient_accumulation_steps": 8,
        "learning_rate": 1e-4,
        "num_epochs": 10,
        "lora_rank": 4,
        "mixed_precision": "fp16",
        "image_size": 512
    }
    print("📋 使用 T4 優化配置")
    
elif "V100" in gpu_name:
    config = {
        "train_batch_size": 2,
        "gradient_accumulation_steps": 4,
        "learning_rate": 1e-4,
        "num_epochs": 15,
        "lora_rank": 8,
        "mixed_precision": "fp16",
        "image_size": 512
    }
    print("📋 使用 V100 優化配置")
    
elif "A100" in gpu_name:
    config = {
        "train_batch_size": 4,
        "gradient_accumulation_steps": 2,
        "learning_rate": 1e-4,
        "num_epochs": 20,
        "lora_rank": 16,
        "mixed_precision": "fp16",
        "image_size": 768
    }
    print("📋 使用 A100 優化配置")
    
else:
    # 通用配置
    config = {
        "train_batch_size": 1,
        "gradient_accumulation_steps": 4,
        "learning_rate": 5e-5,
        "num_epochs": 5,
        "lora_rank": 4,
        "mixed_precision": "fp16",
        "image_size": 512
    }
    print("📋 使用通用配置")

# 顯示配置
print("\n📊 訓練配置:")
for key, value in config.items():
    print(f"   {key}: {value}")

print("\n✅ 配置完成")

## 🚀 步驟 8: 開始訓練

In [ ]:
# 開始訓練
print("🚀 開始 SD v1.5 微調訓練...")
print("這可能需要 30-60 分鐘，請保持 Colab 活躍狀態")

try:
    # 初始化訓練器
    trainer = FashionSDFineTuner(config)
    
    # 提取特徵
    print("\n🔍 提取圖片特徵...")
    captions = trainer.extract_features_from_images(image_paths)
    
    # 準備數據集
    print("\n📊 準備訓練數據...")
    dataloader = trainer.prepare_dataset(image_paths, captions)
    
    # 開始訓練
    print("\n🎯 開始微調訓練...")
    trainer.train(dataloader)
    
    print("\n🎉 訓練完成！")
    
except Exception as e:
    print(f"❌ 訓練失敗: {e}")
    print("\n🔧 建議解決方案:")
    print("1. 檢查 GPU 記憶體是否足夠")
    print("2. 減少 batch_size")
    print("3. 重新啟動運行時並重新執行")
    raise

## 📊 步驟 9: 查看訓練結果

In [ ]:
# 查看訓練結果
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display

print("📊 查看訓練結果...")

# 檢查生成的檔案
model_dir = "models/final_model"
validation_dir = "validation"
logs_dir = "logs"

print(f"\n📁 模型檔案:")
if os.path.exists(model_dir):
    for file in os.listdir(model_dir):
        size = os.path.getsize(os.path.join(model_dir, file)) / 1024 / 1024
        print(f"   {file} ({size:.1f} MB)")
else:
    print("   ❌ 未找到模型檔案")

print(f"\n🖼️ 驗證圖片:")
if os.path.exists(validation_dir):
    validation_images = [f for f in os.listdir(validation_dir) if f.endswith('.png')]
    print(f"   生成了 {len(validation_images)} 張驗證圖片")
    
    # 顯示最新的驗證圖片
    if validation_images:
        latest_image = sorted(validation_images)[-1]
        image_path = os.path.join(validation_dir, latest_image)
        print(f"   最新圖片: {latest_image}")
        display(Image(filename=image_path))
else:
    print("   ❌ 未找到驗證圖片")

print(f"\n📈 訓練圖表:")
if os.path.exists("training_progress.png"):
    print("   ✅ 訓練進度圖表已生成")
    display(Image(filename="training_progress.png"))
else:
    print("   ❌ 未找到訓練圖表")

print("\n✅ 結果查看完成")

## 📦 步驟 10: 打包與下載

In [ ]:
# 打包訓練結果
import zipfile
from datetime import datetime
from google.colab import files

print("📦 打包訓練結果...")

# 創建壓縮檔案
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
package_name = f"sd_v1.5_finetuned_{timestamp}.zip"

with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    
    # 添加模型檔案
    model_dir = "models/final_model"
    if os.path.exists(model_dir):
        for root, dirs, files in os.walk(model_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, "models")
                zipf.write(file_path, f"model/{arcname}")
        print("   ✅ 模型檔案已添加")
    
    # 添加驗證圖片
    validation_dir = "validation"
    if os.path.exists(validation_dir):
        for file in os.listdir(validation_dir):
            if file.endswith('.png'):
                file_path = os.path.join(validation_dir, file)
                zipf.write(file_path, f"validation/{file}")
        print("   ✅ 驗證圖片已添加")
    
    # 添加訓練圖表
    if os.path.exists("training_progress.png"):
        zipf.write("training_progress.png", "training_progress.png")
        print("   ✅ 訓練圖表已添加")
    
    # 添加配置檔案
    config_content = f"""# SD v1.5 Fine-tuning Configuration
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

GPU: {gpu_name}
Total Images: {len(image_paths)}
Training Configuration:
"""
    for key, value in config.items():
        config_content += f"  {key}: {value}\n"
    
    zipf.writestr("config.txt", config_content)
    print("   ✅ 配置檔案已添加")

# 檢查檔案大小
file_size = os.path.getsize(package_name) / 1024 / 1024
print(f"\n📦 打包完成: {package_name} ({file_size:.1f} MB)")

# 下載檔案
print("📥 開始下載...")
files.download(package_name)

print("\n🎉 所有步驟完成！")
print("\n📋 訓練摘要:")
print(f"   • 訓練圖片: {len(image_paths)} 張")
print(f"   • GPU 類型: {gpu_name}")
print(f"   • 訓練輪數: {config['num_epochs']}")
print(f"   • 模型已保存並打包下載")
print(f"   • 檔案大小: {file_size:.1f} MB")

## 🔧 故障排除

### 常見問題

1. **GPU 記憶體不足**
   - 減少 `train_batch_size` 到 1
   - 增加 `gradient_accumulation_steps`
   - 使用較小的 `image_size`

2. **依賴衝突**
   - 重新啟動運行時
   - 重新執行依賴安裝步驟

3. **訓練中斷**
   - 檢查 Colab 會話是否過期
   - 重新連接並從檢查點恢復

4. **上傳失敗**
   - 檢查網路連接
   - 確保圖片格式正確
   - 重新上傳圖片

### 獲得幫助
如果遇到其他問題，請查看相關文檔或聯繫支援。